# OpenStreetMap Wrangling with MongoDB

### Justin Rogers

Map Area: Boulder CO, United States  
[Boulder on OpenStreetMaps](http://www.openstreetmap.org/relation/112298)  
[Overpass API Link for Boulder](http://overpass-api.de/api/map?bbox=-105.4142,39.9490,-105.0653,40.1099)

1. [Problems Encountered in the Map](#section1)
2. [Data Overview](#section2)
3. [Additional Ideas](#section2)

## <a id='section1'>Problems Encountered in the Map</a>

## <a id='section2'>Data Overview</a>

## <a id='section3'>Additional Ideas</a>